## Programming Exercise 6: Support Vector Machines
#### Author - Rishabh Jain

In [1]:
import warnings
warnings.simplefilter('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import cvxopt
%matplotlib inline

from scipy.io import loadmat

#### Learning Resources
1. [SVM Video Lecture (MIT)](https://www.youtube.com/watch?v=_PwhiWxHK8o)
2. [29 to 33 SVM Video Lectures (University of Buffalo)](https://www.youtube.com/watch?v=N4pai7eZW_o&list=PLhuJd8bFXYJsSXPMrGlueK6TMPdHubICv&index=29)
3. [Support Vector Machine Succinctly (PDF)](./Lectures/SVM_succinctly.pdf)
4. [An Idiot’s guide to Support vector machines](./Lectures/SVM_notes.pdf)

### Maths Behind SVM

For two-class, such as the one shown below, there are lots of possible linear separators. Intuitively, a decision boundary drawn in the middle of the void between data items of the two classes seems better than one which approaches very close to examples of one or both classes. While some learning methods such as the logistic regression find just any linear separator. **The SVM in particular defines the criterion to be looking for a decision surface that is MAXIMALLY far away from any data point**. This distance from the decision surface to the closest data point determines the margin of the classifier.

<img src="images/svm1.PNG" width="380"/>

Let's imagine a vector $\vec{w}$ perpendicular to the margin and an unknown data point $\vec{u}$ which can be on either side of the margin. In order to know whether $\vec{u}$ is on the right or left side of the margin, we will project (Dot product) $\vec{u}$ onto $\vec{w}$.

$$\vec{w}.\vec{u}\geq c$$
$$\boxed{\vec{w}.\vec{u}+b\geq 0}\;\;(1)$$ 

If the projection of $\vec{u}$ plus some constant $b$ is greater than zero, then its a positive sample otherwise its a negative sample.**Eq. (1) is our DECISION RULE**. Here the problem is that we don't know what $w$ and $b$ to use.  

**An unknown sample may be located anywhere inside or outside the margin (i.e. >0 or <0), but if it's a known positive sample $\vec{x_{+}}$ then the SVM decision rule should insist the dot product plus some constant $b$ to be 1 or greater than 1.** Likewise for a negative sample $\vec{x_{-}}$, dot product plus some constant $b$ should be less than or equal to -1 Hence:

$\vec{w}.\vec{x_{+}}+b\geq 1 $   
$\vec{w}.\vec{x_{-}}+b\leq -1 $ 

Introducing a variable $y_i$ such that :  

$$\begin{equation}
  y_{i}=\begin{cases}
    +1 & \text{for +ve samples}\\
    -1 & \text{for -ve samples}
  \end{cases}
\end{equation}$$

Mutiplying the above two inequality eqauations with $y_i$:

For +ve sample : $y_{i}(\vec{w}.\vec{x_{i}}+b) \geq 1$  
For -ve sample : $y_{i}(\vec{w}.\vec{x_{i}}+b) \geq 1$

###### Note : Sign changed from $\leq$ to $\geq$ because $y_i$ is -1 in case of -ve samples
Since both the equations are same, we can rewrite them as :

$$\boxed{y_{i}(\vec{w}.\vec{x_{i}}+b)\geq 1}\;\;(2)$$

$$\boxed{y_{i}(\vec{w}.\vec{x_{i}}+b)-1= 0}\;\;(3)\;\;\text{For samples on margin}$$

Eq.(2) is basically a **constraint** for our margin, which means that **all the training samples should be on the correct side OR on the margin** (i.e. +ve samples on the right and -ve samples on the left side of the margin) and **NO training sample should be inside the margin at all meaning ZERO TRAINING ERROR.** 

###### Let's calculate the width of the margin.

<img src="images/svm2.PNG" width="400"/>

Let's imagine two vectors $\vec{x_+}$ and $\vec{x_-}$, both are +ve and -ve known samples respectively. The difference of these two vectors is a resultant vector called $\vec{R}$ where :  
$$\vec{R}=\vec{x_+}-\vec{x_-}$$

All we need is a $\hat{u}$, **so that the WIDTH of the margin will be the projection of $\vec{R}$ onto $\hat{u}$**. From the first image, we already know a vector $\vec{w}$ in the same direction.  
$$\hat{u}=\frac{\vec{w}}{||w||}$$  
**WIDTH** $=\vec{R}.\hat{u} $  

$\;\;\;\;\;\;\;\;\;\;=(\vec{x_+}-\vec{x_-}).\frac{\vec{w}}{||w||}$  
$\;\;\;\;\;\;\;\;\;\;=\frac{(\vec{x_+}.\vec{w}-\vec{x_-}.\vec{w})}{||w||}$

Using Eq.(3), we get

$\;\;\;\;\;\;\;\;\;\;=\frac{(1-b+1+b)}{||w||}$
$$\boxed{\text{WIDTH}=\frac{2}{||w||}}\;\;(4)$$

Now, we want to maximize the margin while incurring zero training error.

max $\frac{2}{||w||}$ with 0 loss OR (Flipping for mathematical convenience)

min $\frac{||w||}{2}\;$ with 0 loss OR (Squaring the numerator for mathematical convenience)

min $\frac{||w||^2}{2}$ with 0 loss **(NO LONGER AN UNCONSTRAINED OPTIMIZATION)**

>Note: 
>* Lagrange Multipliers are explained [**here**](#Understanding-Lagrange-Multipliers).
>* Primal and Dual formulations are explained [**here**](#Primal-and-Dual-Formulations).

### Hard-Margin SVM Classifier

#### Optimization Formulation
$$\boxed{
\min_{w,b}\;\;\frac{||w||^2}{2}\\  
\text{s.t.}\;\;y_{i}(w^{T}x_{i}+b)\geq 1,\;\;\forall{i}
}\;\;(5)$$

In order to solve a constrained optimization problem, Lagrange multipliers are used.  

Since the Objective function is convex (parabola) and all the Constraints are affine (linear) too. Solving dual or primal, answer is going to be same. Rewriting above constrained optimization problem as Lagrangian:

$$\boxed{\min_{w,b,\alpha} L(w,b,\alpha)=\frac{||w||^2}{2}+\sum_{i=1}^{N}\alpha_{i}(1-y_{i}(w^{T}x_{i}+b))\\
\text{s.t.}\;\;\alpha_{i}\geq0,\;\;\forall{i}}\;\;(6)$$

Rewriting above Lagrangian function as a Dual lagrangian:

$$\max_{\alpha} \min_{w,b}(Lw,b,\alpha)\\
\text{s.t.}\;\;\alpha_{i}\geq0,\;\;\forall{i}$$

**OK, let's first minimize the $L(w,b,\alpha)$ w.r.t. $w,b$:**

$$\min_{w,b} L(w,b,\alpha)$$

$\frac{\partial}{\partial{w}}L(w,b,\alpha)=w-\sum_{i=1}^{N}\alpha_{i}y_{i}x_{i}$

>Setting $\frac{\partial}{\partial{w}}L(w,b,\alpha)=0$ gives us $\boxed{w=\sum_{i=1}^{N}\alpha_{i}y_{i}x_{i}}\;\;(7)$

$\frac{\partial}{\partial{b}}L(w,b,\alpha)=-\sum_{i=1}^{N}\alpha_{i}y_{i}$

>Setting $\frac{\partial}{\partial{b}}L(w,b,\alpha)=0$ gives us $\boxed{\sum_{i=1}^{N}\alpha_{i}y_{i}=0}\;\;(8)$

We now will take Eq.(7) and Eq.(8) and plug them back into our full lagrangian Eq.(6) to get a reduced lagrangian that depends only on $\alpha$:

$\begin{align*}
L(w,b,\alpha)&=\frac{w^Tw}{2}+\sum_{i=1}^{N}\alpha_{i}-\sum_{i=1}^{N}\alpha_{i}y_{i}w^{T}x_{i}-\sum_{i=1}^{N}\alpha_{i}y_{i}b\\
&=\frac{1}{2}(\sum_{i=1}^{N}\alpha_{i}y_{i}x_{i})^T(\sum_{j=1}^{N}\alpha_{j}y_{j}x_{j})+\sum_{i=1}^{N}\alpha_{i}-\sum_{i=1}^{N}\alpha_{i}y_{i}(\sum_{j=1}^{N}\alpha_{j}y_{j}x_{j})^{T}x_{i}-0\\
&=\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}(\alpha_{i}\alpha_{j}y_{i}y_{j}x_{i}^{T}x_{j})+\sum_{i=1}^{N}\alpha_{i}-\sum_{i=1}^{N}\sum_{j=1}^{N}(\alpha_{i}\alpha_{j}y_{i}y_{j}x_{i}^{T}x_{j})\\
&=\sum_{i=1}^{N}\alpha_{i}-\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}(\alpha_{i}\alpha_{j}y_{i}y_{j}x_{i}^{T}x_{j})
\end{align*}$

**Above equation is free of any $w$ and $b$. Now, let's maximize the $L(w,b,\alpha)$ w.r.t. $\alpha$:**

$$\boxed{\max_{\alpha} L(w,b,\alpha)=\sum_{i=1}^{N}\alpha_{i}-\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}(\boxed{\alpha_{i}\alpha_{j}}y_{i}y_{j}x_{i}^{T}x_{j})\\
\text{s.t. }\alpha_{i}\geq0\;\;\forall{i}\\
\text{and }\sum_{i=1}^{N}\alpha_{i}y_{i}=0\;\;\forall{i}}\;\;(9)$$

### Soft-Margin SVM Classifier

#### What if the data is not linearly separable??
If the data is not linearly separable, Hard-Margin SVM will fail to fit the data as it tries to incur zero training error. Here, incurring zero training error is not possible but we can still learn a maximum margin hyperplane if we:

<img src="images/svm3.PNG" width="380"/>

- Relax the constraint by introducing **slack variable** $(\xi_{i})$
- Allow some examples to be **misclassified**. For misclassification, $(\xi_{i}>1)$
- Allow some examples to fall inside the margin
- Minimize the number of such examples (Ensuring not too many points are on the wrong side of margin)
$$\min C\sum_{i=1}^N\xi_{i}$$ 
Where **C controls the impact of the margin and the margin error.**

#### Optimization Formulation
$$\boxed{
\min_{w,b}f(w,b)=\frac{||w||^{2}}{2}+C\sum_{i=1}^{N}\xi_{i}\\
\text{s.t.}\;\;\;y_{i}(w^{T}x_{i}+b)\geq1-\xi_{i},\;\;\forall{i}\\
\;\;\;\;\;\;\;\;\xi_{i}\geq0,\;\;\forall{i}
}\;\;(12)$$

Rewriting above constrained optimization problem as Lagrangian:

$$\boxed{\min_{w,b,\xi,\alpha,\beta} L(w,b,\xi,\alpha,\beta)=\frac{||w||^2}{2}+C\sum_{i=1}^{N}\xi{i}+\sum_{i=1}^{N}\alpha_{i}[(1-\xi_{i})-y_{i}(w^{T}x_{i}+b)]+\sum_{i=1}^{N}\beta_{i}(-\xi_{i})\\
\text{s.t.}\;\;\alpha_{i}\geq0,\beta_{i}\geq0,\;\;\forall{i}}\;\;(13)$$

Rewriting above Lagrangian function as a Dual lagrangian:

$$\max_{\alpha\beta} \min_{w,b,\xi}L(w,b,\xi)\\
\text{s.t.}\;\;\alpha_{i}\geq0,\beta_{i}\geq0,\;\;\forall{i}$$

**OK, let's first minimize the $L(w,b,\xi,\alpha)$ w.r.t. $w,b$ and $\xi$ :**

$$\min_{w,b,\xi} L(w,b,\xi,\alpha,\beta)$$

$\frac{\partial}{\partial{w}}L(w,b,\xi,\alpha,\beta)=w-\sum_{i=1}^{N}\alpha_{i}y_{i}x_{i}$

>Setting $\frac{\partial}{\partial{w}}L(w,b,\xi,\alpha,\beta)=0$ gives us $\boxed{w=\sum_{i=1}^{N}\alpha_{i}y_{i}x_{i}}\;\;\text{Same as Eq.(7)}$

$\frac{\partial}{\partial{b}}L(w,b,\xi,\alpha,\beta)=-\sum_{i=1}^{N}\alpha_{i}y_{i}$

>Setting $\frac{\partial}{\partial{b}}L(w,b,\xi,\alpha,\beta)=0$ gives us $\boxed{\sum_{i=1}^{N}\alpha_{i}y_{i}=0}\;\;\text{Same as Eq.(8)}$

$\frac{\partial}{\partial{\xi}}L(w,b,\xi,\alpha,\beta)=C-\alpha_{i}-\beta_{i}$

>$\begin{align*}
\text{Setting }&\frac{\partial}{\partial{\xi}}L(w,b,\xi,\alpha,\beta)=0\\
&=>\boxed{C-\alpha_{i}-\beta_{i}=0}\;\;(14)\\
&=>\beta_{i}=C-\alpha_{i}\\\\
&\text{But because the }\beta_{i}\text{'s are a dual variables with }\beta_{i}\geq0\text{, then this leads to:}\\
&=>C-\alpha_{i}\geq0\\
&=>\alpha_{i}\leq C\\\\
&\text{This along with the fact that }\alpha_{i}\text{ are dual variables with }\alpha_{i}\geq0\text{ we have:}\\
&\boxed{0\leq\alpha_{i}\leq C}\;\;(15)
\end{align*}$

We now will take these results and plug them back into our full lagrangian to get a reduced lagrangian that depends only on $\alpha$ and $\beta$:

Replacing $w$ using Eq.(7) in Eq.(13), we get

$\begin{align*}
L(w,b,\xi,\alpha,\beta)&=\frac{1}{2}(\sum_{i=1}^{N}\alpha_{i}y_{i}x_{i})^{T}(\sum_{i=1}^{N}\alpha_{i}y_{i}x_{i})+C\sum_{i=1}^{N}\xi_{i}+\sum_{i=1}^{N}\alpha_{i}[(1-\xi_{i})-y_{i}((\sum_{j=1}^{N}\alpha_{j}y_{j}x_{j})^{T}x_{i}+b)]-\sum_{i=1}^{N}\beta_{i}\xi_{i}\\
&=\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}(\alpha_{i}\alpha_{j}y_{i}y_{j}x_{i}^{T}x_{j})+C\sum_{i=1}^{N}\xi_{i}+\sum_{i=1}^{N}\alpha_{i}[(1-\xi_{i})-(\sum_{j=1}^{N}\alpha_{j}y_{i}y_{j}x_{j})^{T}x_{i}+b]-\sum_{i=1}^{N}\beta_{i}\xi_{i}\\
&=\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}(\alpha_{i}\alpha_{j}y_{i}y_{j}x_{i}^{T}x_{j})+C\sum_{i=1}^{N}\xi_{i}+\sum_{i=1}^{N}\alpha_{i}-\sum_{i=1}^{N}\alpha_{i}\xi_{i}-\sum_{i=1}^{N}\sum_{j=1}^{N}\alpha_{i}\alpha_{j}y_{i}y_{j}x_{i}^{T}x_{j}+b\sum_{i=1}^{N}\alpha_{i}y_{i}-\sum_{i=1}^{N}\beta_{i}\xi_{i}\\
&=-\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}(\alpha_{i}\alpha_{j}y_{i}y_{j}x_{i}^{T}x_{j})+C\sum_{i=1}^{N}\xi_{i}+\sum_{i=1}^{N}\alpha_{i}-\sum_{i=1}^{N}\alpha_{i}\xi_{i}+b\sum_{i=1}^{N}\alpha_{i}y_{i}-\sum_{i=1}^{N}\beta_{i}\xi_{i}\\
&=-\frac{1}{2}
\sum_{i=1}^{N}\sum_{j=1}^{N}(\alpha_{i}\alpha_{j}y_{i}y_{j}x_{i}^{T}x_{j})+C\sum_{i=1}^{N}\xi_{i}+\sum_{i=1}^{N}\alpha_{i}-(\sum_{i=1}^{N}\alpha_{i}+\sum_{i=1}^{N}\beta_{i})\xi_{i}\;\;\text{Using Eq.(8)}\\
&=-\frac{1}{2}
\sum_{i=1}^{N}\sum_{j=1}^{N}(\alpha_{i}\alpha_{j}y_{i}y_{j}x_{i}^{T}x_{j})+C\sum_{i=1}^{N}\xi_{i}+\sum_{i=1}^{N}\alpha_{i}-C\sum_{i=1}^{N}\xi_{i}\;\;\text{Using Eq.(14)}\\
&=\sum_{i=1}^{N}\alpha_{i}-\frac{1}{2}
\sum_{i=1}^{N}\sum_{j=1}^{N}(\alpha_{i}\alpha_{j}y_{i}y_{j}x_{i}^{T}x_{j})
\end{align*}$

**Above equation is free of any $w,b,\xi$ and $\beta$. Now, let's maximize the $L(w,b,\xi,\alpha,\beta)$ w.r.t. $\alpha$:**

$$\boxed{\max_{\alpha} L(w,b,\alpha)=\sum_{i=1}^{N}\alpha_{i}-\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}(\boxed{\alpha_{i}\alpha_{j}}y_{i}y_{j}x_{i}^{T}x_{j})\\
\text{s.t. }0\leq\alpha_{i}\leq C\;\;\forall{i}\\
\text{and }\sum_{i=1}^{N}\alpha_{i}y_{i}=0\;\;\forall{i}}\;\;(16)$$


### KKT conditions (Hard & Soft SVM)

Eq.(9) & Eq.(16) is Quadratic constraint optimization problem because two unknowns $\alpha_j$ & $\alpha_j$ are getting multiplied together. **Eq.(9) is always solved using some QP solver (like cvxopt in python).**

Let $\alpha^{*}_{1},\alpha^{*}_{2},...\alpha^{*}_{N}$ be the solution of QP (Quadratic Programming) problem.

1. $\frac{\partial}{\partial{w}}L(w,b,\alpha)=0,\;\;\;\;$we got Eq.(7)
2. $\frac{\partial}{\partial{b}}L(w,b,\alpha)=0,\;\;\;\;$we got Eq.(8)
3. $y_{i}(w^{T}x_{i}+b)-1\geq 0\;\;\;$Constraints
4. $\alpha_{i}(y_{i}(w^TX_{i}+b)-1)=0$
5. $\alpha_{i}\geq0$

Using KKT condition 4 & 5, we can imply that:

- If $\alpha^{*}_{i}=0\;\;$ then $\;\;y_{i}(w^TX_{i}+b)-1\geq0$
- If $\alpha^{*}_{i}>0\;\;$ then $\;\;y_{i}(w^TX_{i}+b)-1=0\;\;$($x$ is on the margin)

**Only train examples that lie on the margin are relevant. These are called SUPPORT VECTORS.**

$$\boxed{w=\sum_{i=1}^{N}\alpha^{*}_{i}y_{i}x_{i}}\;\;(10)$$ 

For $\alpha^{*}_{i}>0$,  
$y_{i}(w^TX_{i}+b)-1=0\;\;$

$$\boxed{b=\frac{1}{y_{i}}-w^{T}x_{i}}\;\;(11)$$

All $b$'s are ideally equal otherwise an average is taken.

**Now for a test point $x^{*}$ :** 

$\begin{align*}
y&=w^Tx^*+b\\
&=(\sum_{i=0}^{N}\alpha_{i}^{*}y_{i}x_{i})^{T}x^{*}+b\\  
&=\sum_{\alpha_{i}>0}^{N}\alpha_{i}^{*}y_{i}(x_{i}^{T}x^{*})+b\;\;\text{(Only true for Support Vectors)}\\
\end{align*}$

### Loss function for SVM

If $w^{T}x+b>1$ then $\xi=0$  
else $\xi=1-(w^{T}x+b)$

**Hinge Loss** : $\boxed{J=\sum_{i=1}^{n}max(0,1-y_{i}(w^{T}x+b))}$

### SVM Implementation using CVXOPT Solver

#### Disclaimer
This SVM implementation using CVXOPT is only for educational purpose and will not help you win any MNIST competitions. The sole purpose of this implementation is to apply the mathematics explained above and understand the effect of various kernels.

#### Rewriting the SVM Optimzation problem in CVXOPT format
Since we will solve the optimization problem using CVXOPT library in python, we will need to match the solver's API which, according to the documentation is of the form:

$$\boxed{
\min_{x} \frac{1}{2}x^{T}Px+q^{T}x\\
s.t.\;\;Gx\leq h\\
\;\;\;\;\;\;\;Ax=b
}$$

With api:  
>`cvxopt.solvers.qp(P, q[, G, h[, A, b[, solver[, initvals]]]])`

Let H be a matrix such that $\boxed{H_{i,j}=\sum_{i=1}^{N}\sum_{j=1}^{N}y_{i}y_{j}(x_{i}x_{j})}$, then Eq.(9) becomes:

$$\begin{align*}
\max_{\alpha}&\sum_{i=1}^{N}\alpha_{i}-\frac{1}{2}\alpha^{T}H\alpha\\
s.t.\\
&\alpha_{i}\geq0\\
&\sum_{i=0}^{N}\alpha_{i}y_{i}=0
\end{align*}$$

We convert the sums into vector form and multiply both the objective and the constraint by -1 which turns this into a minimization problem and reverses the inequality :

$$\boxed{
\min_{\alpha}\frac{1}{2}\alpha^{T}H\alpha-1^{T}\alpha\\
\text{s.t.}\;\;-\alpha\leq0\\
\;\;\;\;\;\;\;y^{T}\alpha=0
}$$

We are now ready to convert our numpy arrays into the CVXOPT format, using the same notation as in the documentation this gives:

- P $:=$ H (a matrix of size $m$ x $m$)
- q $:=$ $\vec{-1}$ (a vector of size $m$ x $1$)
- G $:=$ -diag(1) (a diagonal matrix of -1s of size $m$ x $m$)
- h $:=$ $\vec{0}$ (a vector of zeros of size $m$ x $1$)
- A $:=$ $y^{T}$ (the label vector of size $1$ x $m$)
- b $:=$ 0 (a scalar)


In [4]:
class Kernels:
    
    @classmethod
    def linear(cls,x1,x2):
        return np.dot(x1,x2)    

In [81]:
class SVM:
    
    def __init__(self,kernel=Kernels.linear,C=None):
        self.kernel=kernel
        self.C=C
        if self.C is not None:
            self.C=float(self.c)
    
    def fit(self,X,y):
        m,n=X.shape
        # Gram Matrix
        K=np.zeros(shape=(m,m))
        for i in range(m):
            for j in range(m):
                K[i,j]=self.kernel(X[i],X[j])
        H=np.multiply(np.outer(y,y),K)
        P=cvxopt.matrix(H)
        q=cvxopt.matrix(np.ones(m)*-1)
        # For Hard Margin
        if self.C is None:
            G=cvxopt.matrix(np.diag(np.ones(m)*-1))
            h=cvxopt.matrix(np.zeros((m,1)))
        # For Soft Margin
        else:
            pass
        A=cvxopt.matrix(y.astype(float),(1,m))
        b=cvxopt.matrix(0.0)
        # Solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)

In [82]:
X=np.array([[1,2,3,4],[3,4,5,6],[1,2,4,5]])
y=np.array([1,2,4])

svm=SVM()
svm.fit(X,y)

     pcost       dcost       gap    pres   dres
 0: -1.6952e-01 -3.8466e-01  4e+00  2e+00  1e+00
 1: -2.0068e-01 -2.2979e-01  6e-01  3e-01  2e-01
 2:  1.4264e-02 -3.9847e-03  1e-01  3e-02  2e-02
 3: -7.5013e-04 -1.3848e-06  4e-03  1e-03  7e-04
 4: -7.5107e-06 -1.3907e-10  4e-05  1e-05  7e-06
 5: -7.5108e-08 -1.3907e-14  4e-07  1e-07  7e-08
 6: -7.5108e-10 -1.3907e-18  4e-09  1e-09  7e-10
Optimal solution found.


### 1&nbsp;&nbsp;&nbsp;&nbsp;Support Vector Machines

In the first half of the exercise, we will be using SVMs with various examples 2D datasets. And in the next half of the exercise, we will be using SVM to build a spam classifier.

#### 1.1&nbsp;&nbsp;&nbsp;&nbsp;Example Dataset 1

In [74]:
np.multiply((np.ones((5,5))),10)

array([[10., 10., 10., 10., 10.],
       [10., 10., 10., 10., 10.],
       [10., 10., 10., 10., 10.],
       [10., 10., 10., 10., 10.],
       [10., 10., 10., 10., 10.]])

### Understanding Lagrange Multipliers
Lagrange multipliers is a strategy of finding the local maxima and minima of a function subject to **equality** constraints. Let's try to solve a constrained opitimization problem :

#### Example 1 (Equality Constraint)

>minimize $\;\;f(x,y)=2-x^2-2y^2$  
>subject to $\;\;h(x,y)=x+y-1=0$
>
>**We introduce a new variable ($\beta$) called a Lagrange multiplier and study the Lagrange function defined by:**
>
>$$\boxed{L(x,y,\beta)=f(x,y)-\beta h(x,y)}$$
>
>$L(x,y,\beta)=(2-x^2-2y^2)-\beta(x+y-1)$
>
>Now we solve the above equation like an unconstrained optimization problem by taking partial derivatives w.r.t $x$ & $y$ and set them equal to zero solving for $x$, $y$ and $\beta$
>
>$\frac{\partial{L}}{\partial{x}}=0\;\;=>\;\;-2x-\beta=0\;\;=>\;\;x=\frac{-\beta}{2}$
>
>$\frac{\partial{L}}{\partial{y}}=0\;\;=>\;\;-4y-\beta=0\;\;=>\;\;y=\frac{-\beta}{4}$
>
>$\frac{\partial{L}}{\partial{\beta}}=0\;\;=>\;\;x+y-1=0\;\;=>\;\;\beta=\frac{-4}{3}$
>
>$\boxed{x=\frac{4}{6},y=\frac{4}{12},\beta=\frac{-4}{3}}$

#### Example 2 (Inequality Constraints / Karush-Kuhn-Tucker (KKT) conditions)

>maximize $\;\;f(x,y)=3x+4y$  
>subject to $\;\;h_{1}(x,y)=x^2+y^2\leq4$  
>$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;h_{2}(x,y)=x\geq1$
>
>**Note: Inequality constraints should be in the form of $h(x,y)\leq0$**
>
>$$\boxed{L(x,y,\alpha_1,\alpha_2)=f(x,y)-\alpha_1 h_{1}(x,y)-\alpha_2 h_{2}(x,y)\\\;\;\text{s.t. }\alpha_1,\alpha_2\geq0}$$
>
>$L(x,y,\alpha_1,\alpha_2)=3x+4y-\alpha_1(x^2+y^2-4)-\alpha_2(-x+1)$  
>
>**KKT Conditions :**
>
>1. $\frac{\partial{L}}{\partial{x}}=3-2\alpha_1x+\alpha_2=0$
>
>2. $\frac{\partial{L}}{\partial{y}}=4-2\alpha_1y=0$
>
>3. $\alpha_1(x^2+y^2-4)=0$
>
>4. $\alpha_2(-x+1)=0$
>
>5. $\alpha_1,\alpha_2\geq0$ 
>
>A constraint is considered to be binding (active) if changing it also changes the optimal solution. Less severe constraints that do not affect the optimal solution are non-binding (non active). For 2 constraints possible combinations are :
>
>- Both constraints are binding
>- Constraint 1 binding, Constraint 2 not binding
>- Constraint 2 binding, Constraing 1 not binding
>- Both constraints are not binding
>
>**POSSIBILITY 1 : Both constraints are binding**
>
>$-x+1=0\;\text{and}\;\alpha_2>0\;\;=>\;\;x=1$  
>$x^2+y^2-4=0\;\text{and}\;\alpha_1>0\;\;=>\;\;x^2+y^2=4\;\;=>\;\;1+y^2=4\;\;=>\;\;y=\pm\sqrt{3}$  
>
>(a) For $y=+\sqrt{3}$ 
>
>>Condition 2 becomes:  
>>$4-2\sqrt{3}\alpha_1=0\;\;=>\;\;\alpha_1=\frac{2}{\sqrt{3}}>0$  
>>Condition 1 becomes:  
>>$3-2\alpha_1+\alpha_2=0\;\;=>\;\;3-\frac{4}{\sqrt{3}}+\alpha_2=0\;\;=>\;\;\alpha_2=\frac{4}{\sqrt{3}}-3<0$ (KKT condition fails)
>
>(a) For $y=-\sqrt{3}$  
>
>>Condition 2 becomes:  
>>$4+2\sqrt{3}\alpha_1=0\;\;=>\;\;\alpha_1=\frac{-2}{\sqrt{3}}<0$ (KKT condition fails)    
>>Condition 1 becomes:  
>>$3-2\alpha_1+\alpha_2=0\;\;=>\;\;3+\frac{4}{\sqrt{3}}+\alpha_2=0\;\;=>\;\;\alpha_2=\frac{-4}{\sqrt{3}}-3<0$ (KKT condition fails)
>
>**POSSIBILITY 2 : Constraint 1 binding , Contraint 2 not binding**
>
>$x>1\;\text{and}\;\boxed{\alpha_2=0}$  
>$x^2+y^2<4\;\text{and}\;\alpha_1>0\;\;=>\;\;x=+\sqrt{4-y^{2}}$  
>
>>Condition 1 becomes:  
>>$3-2\alpha_1x=0\;\;=>\;\;x=\frac{3}{2\alpha_1}\;\;=>\;\;3-2\alpha_1\sqrt{4-y^{2}}=0\;\;=>\;\;\alpha_1=\frac{3}{2\sqrt{4-y^{2}}}$  
>>Condition 2 becomes:  
>>$4-2\alpha_1y=0\;\;=>\;\;4-\frac{3y}{\sqrt{4-y^{2}}}=0\;\;=>\;\;4\sqrt{4-y^{2}}=3y\;\;=>\;\;16(4-y^2)=9y^2\;\;=>\;\;64-16y^2=9y^2\;\;=>\;\;64=25y^2\;\;=>\;\;y=\pm\frac{8}{5}$
>
>$\boxed{\alpha_1=\frac{3}{2\sqrt{4-\frac{64}{25}}}=\frac{3}{2(\frac{6}{5})}=\frac{5}{4}>0}$  
>$x=+\sqrt{4-y^{2}}\;\;=>\;\;x=\frac{6}{5}$
>
>1 candidate point: $\boxed{(x,y)=(\frac{6}{5},\frac{8}{5})}$
>
>**POSSIBILITY 3 : Constraint 2 binding , Contraint 1 not binding**
>
>$x=1\;\text{and}\;\alpha_2>0$  
>$x^2+y^2<4\;\text{and}\;\alpha_1=0$  
>
>>Condition 2 becomes:  
>>$4-2\alpha_1y=0\;\;=>\;\;4=0$ (Contradiction, no candidate points)  
>
>**POSSIBILITY 4 : Both constraints are not binding**
>
>$x>1\;\text{and}\;\alpha_2=0$  
>$x^2+y^2<4\;\text{and}\;\alpha_1=0$  
>
>>Condition 2 becomes:  
>>$4-2\alpha_1y=0\;\;=>\;\;4=0$ (Contradiction, no candidate points)  
>
>**Check maximality of the candidate point :**
>
>$f(\frac{6}{5},\frac{8}{5})=3(\frac{6}{4})+4(\frac{8}{5})=\frac{18}{5}+\frac{32}{5}=10$
>
>Optimal Solution : $\boxed{x=\frac{6}{5},y=\frac{8}{5},\alpha_1=0,\alpha_2=\frac{5}{4}}$

#### Handling both types of Constraints

$$\boxed{
\min_{w}\;\;f(w)\\
\text{subject to}\;\;g_{i}(w)\leq0\;\;\;i=1,2,...k\\
\text{and}\;\;\;\;\;\;\;\;\;\;h_{i}(w)=0\;\;\;i=1,2,...l\\
}$$

**Generalized Lagrangian**  
$$\boxed{
L(w,\alpha,\beta)=f(w)+\sum_{i=1}^{k}\alpha_{i}g_{i}(w)+\sum_{i=1}^{l}\beta_{i}h_{i}(w)\\
\text{subject to}\;\;\alpha_{i}\geq0,\forall_i
}$$

### Primal and Dual Formulations

#### Primal Optimization

Let $\theta_p$ be defined as :
$$\boxed{\theta_p(w)=\max_{\alpha,\beta;\alpha_i\geq0}L(w,\alpha,\beta)}$$

Original constrained problem is same as :
$$\boxed{p^*=\min_{w}\theta_P(w)=\min_{w}\max_{\alpha,\beta;\alpha_i\geq0}L(w,\alpha,\beta)}$$

Solving $p^*$ is same as solving the constrained optimization problem.

$$\begin{equation}
  \theta_{p}(w)=\begin{cases}
    f(w) & \text{if all constraints are satifsied}\\
    \infty & \text{else}
  \end{cases}
\end{equation}$$

#### Dual Optimization
Let $\theta_d$ be defined as :
$$\boxed{\theta_d(w)=\min_{w}L(w,\alpha,\beta)}$$

Original constrained problem is same as :
$$\boxed{d^*=\max_{\alpha,\beta;\alpha_i\geq0}\theta_d(w)=\max_{\alpha,\beta;\alpha_i\geq0}\min_{w}L(w,\alpha,\beta)}$$

#### Week Duality Theorem (Min-Max Theorem)

$$\boxed{d^{*}\leq p^{*}}$$

Both of them are equal ($d^{*}=p^{*}$) when
- f(w) is convex
- Constraints are affine (Linear)

#### Relation between Primal and Dual

+ In genral $d^*\leq p^*$, for SVM optimization the equality holds true.
+ Certain conditions should be true.
+ Known as the **Kahrun-Kuhn-Tucker (KKT)** conditions.
+ For $d^*=p^*=L(w^*,\alpha^*,\beta^*)$ :
>+ $\frac{\partial}{\partial{w}}L(w^*,\alpha^*,\beta^*)=0$   
>+ $\frac{\partial}{\partial{\beta_{i}}}L(w^*,\alpha^*,\beta^*)=0\;\;\;\;\;\;\;i=1,2,...l$  
>+ $\alpha_{i}g_{i}(w^{*})=0\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;i=1,2,...k$  
>+ $g_{i}(w^{*})\leq0\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;i=1,2,...k$  
>+ $\alpha_{i}\geq0\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;i=1,2,...k$